In [1]:
import os

import pandas as pd
import numpy as np

import time

# need to filter warnings related to spacy lemmatizer
import logging
logger = logging.getLogger("spacy")
logger.setLevel(logging.ERROR)

In [2]:
os.chdir('../..')

In [3]:
from src.data.dataload import load_sst
from src.data.dataload import load_agnews

In [4]:
from src.data.perturbations import add_perturbations
from src.data.perturbations import \
    remove_commas, \
    remove_all_punctuation, \
    switch_gender, \
    strip_trailing_punct, \
    add_typo, \
    change_first_name, \
    change_last_name, \
    change_location, \
    contraction, \
    swap_adjectives

## Load datasets

In [5]:
sst = load_sst()
agnews = load_agnews()

In [6]:
input_data = {'sst': sst, 'agnews': agnews}
datasets = list(input_data.keys())

In [7]:
train = {}
dev = {}
test = {}

for dataset in datasets:
    train[dataset], dev[dataset], test[dataset] = input_data[dataset].train_val_test

Using custom data configuration default
Reusing dataset ag_news (/Users/stevengeorge/.cache/huggingface/datasets/ag_news/default/0.0.0/17ec33e23df9e89565131f989e0fdf78b0cc4672337b582da83fc3c9f79fe34d)


In [8]:
for dataset in datasets:
    print(train[dataset].shape, dev[dataset].shape, test[dataset].shape)

(8544, 2) (1101, 2) (2210, 2)
(108000, 3) (12000, 3) (7600, 3)


In [9]:
train['agnews'].head()

,sentence,label,title
0,"Reuters - Short-sellers, Wall Street's dwindli...",2,Wall St. Bears Claw Back Into the Black (Reuters)
1,Reuters - Private investment firm Carlyle Grou...,2,Carlyle Looks Toward Commercial Aerospace (Reu...
2,Reuters - Soaring crude prices plus worriesabo...,2,Oil and Economy Cloud Stocks' Outlook (Reuters)
3,Reuters - Authorities have halted oil exportfl...,2,Iraq Halts Oil Exports from Main Southern Pipe...
4,"AFP - Tearaway world oil prices, toppling reco...",2,"Oil prices soar to all-time record, posing new..."


In [10]:
np.random.seed(3)
train_subset = train['sst'].copy().sample(n=100)

In [11]:
train_subset.head()

,sentence,label
6341,"Schnieder bounces around with limp wrists , we...",1
2119,It is an indelible epic American story about t...,2
5105,"Mr. Goyer 's loose , unaccountable direction i...",0
1594,Director Nancy Savoca 's no-frills record of a...,4
387,An average coming-of-age tale elevated by the ...,2


In [15]:
%%time

train_perturbations = add_perturbations(
    df=train_subset, 
    sentence_col_name='sentence', 
    perturbation_functions=[
        remove_commas,
        remove_all_punctuation,
        switch_gender,
        strip_trailing_punct,
        add_typo,
        change_first_name,
        change_last_name,
        change_location,
        contraction,
        swap_adjectives
    ]
)

ValueError: empty range for randrange() (1,1, 0)

In [16]:
train_perturbations.head()

,sentence,label,sentence_tokens,sentence_remove_commas_concat,sentence_remove_commas_tokens,success_remove_commas
0,"Schnieder bounces around with limp wrists , we...",1,"[Schnieder, bounces, around, with, limp, wrist...",Schnieder bounces around with limp wrists wea...,"[Schnieder, bounces, around, with, limp, wrist...","[1, [[6, 14]]]"
1,It is an indelible epic American story about t...,2,"[It, is, an, indelible, epic, American, story,...",It is an indelible epic American story about t...,"[It, is, an, indelible, epic, American, story,...","[1, [[10, 16]]]"
2,"Mr. Goyer 's loose , unaccountable direction i...",0,"[Mr., Goyer, 's, loose, ,, unaccountable, dire...",Mr. Goyer 's loose unaccountable direction is...,"[Mr., Goyer, 's, loose, , unaccountable, direc...","[1, [[4]]]"
3,Director Nancy Savoca 's no-frills record of a...,4,"[Director, Nancy, Savoca, 's, no, -, frills, r...",Director Nancy Savoca 's no-frills record of a...,"[Director, Nancy, Savoca, 's, no, -, frills, r...",0
4,An average coming-of-age tale elevated by the ...,2,"[An, average, coming, -, of, -, age, tale, ele...",An average coming-of-age tale elevated by the ...,"[An, average, coming, -, of, -, age, tale, ele...",0


In [25]:
(train_perturbations['sentence_tokens'].apply(len) != train_perturbations['sentence_swap_adj_tokens'].apply(len)).sum()

0

In [19]:
train_perturbations['sentence_change_last_name_tokens'].apply(len)

0      39
1      44
2      41
3      19
4       8
       ..
995    39
996    20
997    26
998    30
999    24
Name: sentence_change_last_name_tokens, Length: 1000, dtype: int64